In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim
from gensim import corpora, models, similarities
import os
import utility_synopsis
import math
import numpy as np
from numpy import linalg as la
from scipy.spatial.distance import cosine
import pandas as pd
import matplotlib.pyplot as plt

from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf
import os
import random
import datetime
#from surprise import BaselineOnly
#from surprise import Dataset
#from surprise import evaluate
from surprise import Reader
import collections as coll

import cPickle as pickle

In [ ]:
count_limit = 1000 # only calculate neighbors of a subset of users to avoid space and time problems

def extract_item_info(filename, delimiter, genre_delimiter):
    data = {}

    with open(filename , 'r') as f:
        for i, line in enumerate(f):
            itemId, title, genre_list = map(lambda x: x.strip(), line.split(delimiter))
            
            data[itemId] = (title, genre_list.split(genre_delimiter))
    return data

item_file_name, item_file_delimiter, genre_delimiter = os.path.sep.join(['ml-1m', 'movies.dat']), '::', '|'
item_info = extract_item_info(item_file_name, item_file_delimiter, genre_delimiter)

###
def extract_user_item_interaction(filename, delimiter):
    data = {}

    with open(filename , 'r') as f:
        for i, line in enumerate(f):
            userId, movieId, rating, timestamp = line.split(delimiter)
            #userId = int(userId)
            #movieId = int(movieId)
            rating = float(rating)
            timestamp = int(timestamp)

            if userId not in data:
                data[userId] = []
            data[userId].append((movieId, rating, timestamp))
    
    # order by time
    for userId in data:
        data[userId].sort(key=lambda x: x[2]) 
    return data

rating_file_name, rating_file_delimiter = os.path.sep.join(['ml-1m', 'ratings.dat']), '::'
user_item_interaction = extract_user_item_interaction(rating_file_name, rating_file_delimiter)


In [6]:
# preparation for word2vec approach

model_path = '/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model'
model = gensim.models.Word2Vec.load(model_path)

def user_history2user_repr__simple_average(model, target_user_history): # target_user_history: It should_be_a_list_of_tuples_included_items.
    #print 'target_user_history:', target_user_history
    items_existed_in_model = filter(lambda x: x[0] in model, target_user_history)
    #print 'items_existed_in_model:', items_existed_in_model[0]
    items_translated_to_vecs = map(lambda x: model[x[0]], items_existed_in_model)
    #print 'items_translated_to_vecs:', items_translated_to_vecs[0]
    return np.average(items_translated_to_vecs, axis=0)   

#user_history2user_repr__simple_average(model, user_item_interaction['5989'])
# calculate user representation dict
user_repr__word2vec = {user: user_history2user_repr__simple_average(model, user_item_interaction[user]) for user in user_item_interaction}


2017-02-24 19:45:46,807 : INFO : loading Word2Vec object from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model
2017-02-24 19:45:46,816 : INFO : loading wv recursively from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model.wv.* with mmap=None
2017-02-24 19:45:46,816 : INFO : setting ignored attribute syn0norm to None
2017-02-24 19:45:46,817 : INFO : setting ignored attribute cum_table to None
2017-02-24 19:45:46,817 : INFO : loaded /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model


In [10]:
### calculate neighbors of each user
neighbors__word2vec = {}

for i, target_user in enumerate(user_item_interaction):
    if i == count_limit:
        break
        
    target_repr = user_repr__word2vec[target_user]
    #print target_repr
    simi = [(v, target_repr.dot(user_repr__word2vec[v]) / (la.norm(target_repr * la.norm(user_repr__word2vec[v]))))
            for v in user_item_interaction]
    simi.sort(key=lambda x: -1 * x[1])
    
    #print simi[1:K + 1]
    #print simi[-1 * K:]
    neighbors__word2vec[target_user] = simi

In [ ]:
with open('chap3_exp2__word2vec__user_repr.dump', 'wb') as fout:
    pickle.dump(user_repr__word2vec, fout)

In [11]:
# This might take a long time

with open('chap3_exp2__word2vec__neighbors.dump', 'wb') as fout:
    pickle.dump(neighbors__word2vec, fout)

KeyboardInterrupt: 

In [ ]:
# preparation for simple approach/CF approach


In [ ]:
# preparation for content-based approach


In [13]:
# preparation for svd approach
data_filename, delimiter, data_set = os.path.sep.join(['ml-1m', 'ratings.dat']), '::', '1M'

chap3_exp2_train_file_name = 'chap3_exp2__tmp__svd_train'
chap3_exp2_test_file_name = 'chap3_exp2__tmp__svd_test'
my_sep = ':'

if not os.path.exists(chap3_exp2__svd_dump):
    with open(chap3_exp2_test_file_name, 'w') as f: # Yes, emtpy test file. We are going to use full dataset.
        pass
    with open(chap3_exp2_train_file_name, 'w') as fout: # Write full dataset into train file.
        with open(data_filename, 'r') as f:
            for i, line in enumerate(f):
                fout.write(my_sep.join(line.split(delimiter)))
#

##
reader = Reader(line_format='user item rating timestamp', sep=my_sep)

data = Dataset.load_from_folds([(chap3_exp2_train_file_name, chap3_exp2_test_file_name)], reader=reader)

for trainset, testset in data.folds():
    first_train, first_test = trainset, testset
    
print 'info of train:'
print first_train.n_users
print first_train.n_items
print first_train.n_ratings

print 'info of test:'
print len(first_test)


##
algo = None
chap3_exp2__svd_dump = 'chap3_exp2__svd_dump.algo'
if os.path.exists(chap3_exp2__svd_dump):
    print '[Note]: SVD dump exists already. Going to load it!'
    with open(chap3_exp2__svd_dump, 'rb') as f:
        algo = pickle.load(f)
else:    
    print '[Note]: SVD dump does not exist. Going to train a SVD!'
    # train SVD
    starttime = datetime.datetime.now()

    algo = SVD()
    algo.train(first_train)

    ###
    endtime = datetime.datetime.now()
    interval = (endtime - starttime).seconds
    print 'svd time consumption: %d' % (interval)


##
#all_user_ids_in_svd_train = set()
#with open(train_file_name, 'r') as fin:
#    for i, line in enumerate(fin):
#        userId, movieId, rating, timestamp = line.split(my_sep)
#        all_user_ids_in_svd_train.add(userId)
##


info of train:
6040
3706
1000209
info of test:
0
[Note]: SVD dump does not exist. Going to train a SVD!
svd time consumption: 45


NameError: global name 'inner_iid' is not defined

In [15]:

##
def generate_svd_user_repr(user_item_interaction):
    '''Constraint for @para: There should be some method to extract all user ids from the @para.'''
    user_repr = {}
    svd_missing_count = 0
    for user in user_item_interaction:
        
        try:
            inner_uid = first_train.to_inner_uid(user)
            user_repr[user] = algo.pu[inner_uid]
        except ValueError, e:
            svd_missing_count += 1
    print 'svd_missing_count:', svd_missing_count
    return user_repr

svd_user_repr = generate_svd_user_repr(user_item_interaction)

### calculate neighbors of each user
neighbors__svd = {}

for i, target_user in enumerate(user_item_interaction):
    if i == count_limit:
        break
        
    target_repr = svd_user_repr[target_user]
    #print target_repr
    simi = [(v, target_repr.dot(svd_user_repr[v]) / (la.norm(target_repr * la.norm(svd_user_repr[v]))))
            for v in user_item_interaction]
    simi.sort(key=lambda x: -1 * x[1])
    
    #print simi[1:K + 1]
    #print simi[-1 * K:]
    neighbors__svd[target_user] = simi

svd_missing_count: 0


In [30]:
all_target_users[1]

'5989'

In [35]:
svd_user_repr['5988']

array([ 0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07320169,
        0.07320169,  0.07320169,  0.07320169,  0.07320169,  0.07

In [34]:
svd_user_repr['5989']

array([ 0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04600196,
        0.04600196,  0.04600196,  0.04600196,  0.04600196,  0.04

In [32]:
svd_user_repr['5989'].dot(svd_user_repr['5988']) / (la.norm(svd_user_repr['5989'] * la.norm(svd_user_repr['5988'])))

1.0

In [26]:
algo.pu[1]

array([ 0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02720527,
        0.02720527,  0.02720527,  0.02720527,  0.02720527,  0.02

In [27]:
algo.pu[2]

array([ 0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04645943,
        0.04645943,  0.04645943,  0.04645943,  0.04645943,  0.04

In [28]:
algo.pu[3]

array([ 0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10212395,
        0.10212395,  0.10212395,  0.10212395,  0.10212395,  0.10

In [42]:
def my_cosine_similarity_with_regression(target, b):
    cos = target.dot(b) / (la.norm(target * la.norm(b)))
    if cos > 0.99999 or cos < -0.99999:
        return -1 * abs(target[0] - b[0])  # for consistency: bigger means more similar
    else:
        return cos # bigger means more similar


In [16]:
all_target_users = []
for i, target_user in enumerate(user_item_interaction):
    if i == count_limit:
        break
    all_target_users.append(target_user)
    

In [49]:
# For user, analyze any two neighborhood calculated by simple approach, content-based approach, 
# word2vec approach and svd approach.


def find_out_intersection_size_sum_with_given_limit_list__x(limit_list, all_target_users):
    intersection_size_map = coll.defaultdict(lambda:coll.defaultdict(list))
    union_size_map = coll.defaultdict(lambda:coll.defaultdict(list))

    missing_count__cb = 0
    missing_count__svd = 0

    i = None
    for __step, target_user in enumerate(all_target_users):
        #print item_id
        if 0 == __step % 100:
            print 'progress: %d/%d' % (__step, len(all_target_users))

        # content-based <START>
        #cb_target_repr = item_repr[item_id] # cb: content based
        #sims_cb = [(iid, cb_target_repr.dot(item_repr[iid]) / 
        #            (la.norm(cb_target_repr * la.norm(item_repr[iid])))) for iid in model.wv.vocab]
        ##print sims[0]
        #sims_cb = sorted(sims_cb, key=lambda item: -1 * item[1])
        # content-based <END>
       
    
        # svd <START>
        #        svd_target_repr = svd_item_repr[item_id]
        #        #print 'svd_target_repr:', svd_target_repr
        #        sims_svd = [(iid, svd_target_repr.dot(svd_item_repr[iid]) / 
        #                     (la.norm(svd_target_repr * la.norm(svd_item_repr[iid])))) for iid in all_item_ids_in_svd_train]
        #        sims_svd = sorted(sims_svd, key=lambda item: -1 * item[1])
        #        #print 'sims_svd:', sims_svd
                
        target_repr__svd = svd_user_repr[target_user]
        simi__svd = [(v, my_cosine_similarity_with_regression(target_repr__svd, svd_user_repr[v]))
                for v in user_item_interaction]
        simi__svd.sort(key=lambda x: -1 * x[1])
        neighbors__svd = simi__svd
        ##
        # svd <END>

        # word2vec <START>
        #        ret_by_word2vec = model.most_similar(str(item_id), topn=max(limit_list))
        
        target_repr__word2vec = user_repr__word2vec[target_user]
        simi__word2vec = [(v, target_repr__word2vec.dot(user_repr__word2vec[v]) 
                 / (la.norm(target_repr__word2vec * la.norm(user_repr__word2vec[v]))))
                for v in user_item_interaction]
        simi__word2vec.sort(key=lambda x: -1 * x[1])
        neighbors__word2vec = simi__word2vec
        # word2vec <END>
        
        print 'neighbors__svd:', neighbors__svd[:10], '...', neighbors__svd[-10:]
        print 'neighbors__word2vec:', neighbors__word2vec[:10], '...', neighbors__word2vec[-10:]
        
        tmp__limit = 400
        tmp__svd = [x[0] for x in neighbors__svd[:tmp__limit]]
        tmp__word2vec = [x[0] for x in neighbors__word2vec[:tmp__limit]]
        print len(set(tmp__svd).intersection(tmp__word2vec))
        print len(set(tmp__svd).union(tmp__word2vec))
        break
        
        for limit in limit_list:
            #print(sims[1:limit+1])
            #most_similar_by_cb = [x[0] for x in sims_cb[1:limit+1]]  # Careful! The index range!
            #print most_similar_by_cb

            most_similar_by_svd = [x[0] for x in sims_svd[1:limit+1]]  # Careful! The index range!
            
            most_similar_by_word2vec = [x[0] for x in ret_by_word2vec][:limit]  # Careful! The index range! Most similar items by word2vec don't include target item itself.
            #print most_similar_by_word2vec

            
            #print 'cb:', most_similar_by_cb
            #print 'word2vec:', most_similar_by_word2vec
            #print 'svd:', most_similar_by_svd
            
            #print set(most_similar_by_cb).intersection(set(most_similar_by_word2vec))
            inter = set(most_similar_by_cb).intersection(set(most_similar_by_word2vec))
            intersection_size_map[limit]['cb__word2vec'].append(len(inter))
            union = set(most_similar_by_cb).union(set(most_similar_by_word2vec))
            union_size_map[limit]['cb__word2vec'].append(len(union))
            
            #
            inter = set(most_similar_by_cb).intersection(set(most_similar_by_svd))
            #                              (len(set(most_similar_by_word2vec).intersection(set(most_similar_by_svd))))
            intersection_size_map[limit]['cb__svd'].append(len(inter))
            union = set(most_similar_by_cb).union(set(most_similar_by_svd))
            union_size_map[limit]['cb__svd'].append(len(union))
            
            inter = set(most_similar_by_word2vec).intersection(set(most_similar_by_svd))
            intersection_size_map[limit]['word2vec__svd'].append(len(inter))
            union = set(most_similar_by_word2vec).union(set(most_similar_by_svd))
            union_size_map[limit]['word2vec__svd'].append(len(union))
                          
        #if 0 < len(inter):
        #    print item_id
        if 0 == __step:
            break

    print 'missing_count__cb:', missing_count__cb
    print 'missing_count__svd:', missing_count__svd

    return intersection_size_map, union_size_map

r = find_out_intersection_size_sum_with_given_limit_list__x([500], all_target_users)
#r = find_out_intersection_size_sum_with_given_limit_list__x(range(10, 500, 10))
print r

progress: 0/1000
neighbors__svd: [('5988', -0.0), ('4090', -1.4184216790821824e-05), ('3074', -2.5446335330497893e-05), ('4753', -2.8442956761937088e-05), ('2275', -3.2858889629949761e-05), ('83', -7.2446903137277086e-05), ('466', -9.5574485427143974e-05), ('2416', -9.7966415514808003e-05), ('2014', -0.00011041765816349747), ('4966', -0.00012157335688735282)] ... [('3151', -0.16927314766211182), ('1198', -0.17496766308106282), ('1790', -0.1794772894462951), ('3537', -0.18701478435385538), ('2744', -0.18749106030001611), ('2458', -0.19245506332582168), ('2195', -0.19469324582868119), ('3679', -0.19985060387038639), ('4539', -0.2189815402125761), ('3610', -0.26431945941847512)]
neighbors__word2vec: [('5988', 1.0), ('5718', 0.9841646), ('2174', 0.98250049), ('1532', 0.98213619), ('3240', 0.98212159), ('4452', 0.98195195), ('5998', 0.98069644), ('11', 0.98042107), ('3057', 0.98027587), ('111', 0.98024207)] ... [('4211', 0.80288029), ('5999', 0.79902619), ('3623', 0.79784542), ('5069', 0.79